# Box Concatenate Test

This notebook is to test concatentating lots of separate netcdf files into one netcdf or zarr file. I will test the functionality using the smaller scale tests located in `/swot/SUM05/amf2288/sync-boxes/test_boxes`

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import argopy
import scipy.ndimage as filter
import scipy
import matplotlib
import gsw
from pathlib import Path

import dask
from dask.diagnostics import ProgressBar

In [2]:
def concatenate_netcdf(input_dir: str, first_dim: str, second_dim: str, output_dir: str, output_file: str):
    input_path = Path(input_dir)
    netcdf_files = list(input_path.glob("*.nc"))
    
    # Create the output directory if it doesn't exist
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Open datasets lazily with dask
    datasets = [xr.open_dataset(str(file), chunks={}) for file in netcdf_files]
    
    # Concatenate along the first dimension
    combined_first_dim = xr.concat(datasets, dim=first_dim)
    
    # Rechunk the data to ensure uniform chunk sizes
    combined_rechunked = combined_first_dim.chunk({first_dim: 256, second_dim: 256})  # Adjust chunk sizes as needed
    
    # Save to NetCDF
    output_file_path = output_path / output_file
    with ProgressBar():
        combined_rechunked.to_netcdf(output_file_path, compute=True)

def concatenate_zarr(input_dir: str, first_dim: str, second_dim: str, output_dir: str, output_file: str):
    input_path = Path(input_dir)
    netcdf_files = list(input_path.glob("*.nc"))
    
    # Create the output directory if it doesn't exist
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Open datasets lazily with dask
    datasets = [xr.open_dataset(str(file), chunks={}) for file in netcdf_files]
    
    # Concatenate along the first dimension
    combined_first_dim = xr.concat(datasets, dim=first_dim)
    
    # Rechunk the data to ensure uniform chunk sizes
    combined_rechunked = combined_first_dim.chunk({first_dim: 256, second_dim: 256})  # Adjust chunk sizes as needed
    
    # Save to Zarr
    output_file_path = output_path / output_file
    with ProgressBar():
        combined_rechunked.to_zarr(output_file_path, compute=True)

In [3]:
# Example usage
input_directory = "/swot/SUM05/amf2288/sync-boxes/test_boxes"
output_directory = "/swot/SUM05/amf2288/sync-boxes/test_concat"
output_netcdf = "test_out.nc"
output_zarr = "test_out.zarr"
first_dim = "N_PROF"
second_dim = "PRES_INTERPOLATED"

In [4]:
# Concatenate along both dimensions and save to NetCDF
concatenate_netcdf(input_directory, first_dim, second_dim, output_directory, output_netcdf)

[########################################] | 100% Completed | 1.05 s


In [7]:
# Concatenate along both dimensions and save to Zarr
concatenate_zarr(input_directory, first_dim, second_dim, output_directory, output_zarr)

[########################################] | 100% Completed | 1.14 s


In [6]:
xr.open_dataset('/swot/SUM05/amf2288/sync-boxes/test_concat/test_out.nc')

<xarray.Dataset> Size: 112MB
Dimensions:            (N_PROF: 2334, PRES_INTERPOLATED: 1001)
Coordinates:
    TIME               (N_PROF) datetime64[ns] 19kB ...
    LATITUDE           (N_PROF) float64 19kB ...
    LONGITUDE          (N_PROF) float64 19kB ...
  * PRES_INTERPOLATED  (PRES_INTERPOLATED) int64 8kB 0 2 4 6 ... 1996 1998 2000
  * N_PROF             (N_PROF) int64 19kB 303 312 478 480 ... 2244 244 2251
    N_PROF_NEW         (N_PROF) int64 19kB ...
    MLD                (N_PROF) int64 19kB ...
    month_frac         (N_PROF) float64 19kB ...
    year_frac          (N_PROF) float64 19kB ...
Data variables:
    CT                 (N_PROF, PRES_INTERPOLATED) float64 19MB ...
    CYCLE_NUMBER       (N_PROF) int64 19kB ...
    DATA_MODE          (N_PROF) <U1 9kB ...
    DIRECTION          (N_PROF) <U1 9kB ...
    PLATFORM_NUMBER    (N_PROF) int64 19kB ...
    PRES               (N_PROF, PRES_INTERPOLATED) float32 9MB ...
    PSAL               (N_PROF, PRES_INTERPOLATED) float32 9MB ...
    SA                 (N_PROF, PRES_INTERPOLATED) float64 19MB ...
    SIG0               (N_PROF, PRES_INTERPOLATED) float64 19MB ...
    TEMP               (N_PROF, PRES_INTERPOLATED) float32 9MB ...
    sample_rate        (N_PROF, PRES_INTERPOLATED) float32 9MB ...
    SPICE              (N_PROF, PRES_INTERPOLATED) float64 19MB ...
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://data-argo.ifremer.fr
    Fetched_by:           amf2288
    Fetched_date:         2025/01/03
    Fetched_constraints:  [x=-175.00/-170.00; y=20.00/25.00; z=0.0/2001.0]
    Fetched_uri:          https://data-argo.ifremer.fr/dac/aoml/4900313/49003...
    Processing_history:   [PRES,TEMP,PSAL] real-time and adjusted/delayed var...

In [8]:
xr.open_zarr('/swot/SUM05/amf2288/sync-boxes/test_concat/test_out.zarr')

<xarray.Dataset> Size: 112MB
Dimensions:            (N_PROF: 2334, PRES_INTERPOLATED: 1001)
Coordinates:
    LATITUDE           (N_PROF) float64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
    LONGITUDE          (N_PROF) float64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
    MLD                (N_PROF) int64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
  * N_PROF             (N_PROF) int64 19kB 303 312 478 480 ... 2244 244 2251
    N_PROF_NEW         (N_PROF) int64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
  * PRES_INTERPOLATED  (PRES_INTERPOLATED) int64 8kB 0 2 4 6 ... 1996 1998 2000
    TIME               (N_PROF) datetime64[ns] 19kB dask.array<chunksize=(256,), meta=np.ndarray>
    month_frac         (N_PROF) float64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
    year_frac          (N_PROF) float64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
Data variables:
    CT                 (N_PROF, PRES_INTERPOLATED) float64 19MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    CYCLE_NUMBER       (N_PROF) int64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
    DATA_MODE          (N_PROF) <U1 9kB dask.array<chunksize=(256,), meta=np.ndarray>
    DIRECTION          (N_PROF) <U1 9kB dask.array<chunksize=(256,), meta=np.ndarray>
    PLATFORM_NUMBER    (N_PROF) int64 19kB dask.array<chunksize=(256,), meta=np.ndarray>
    PRES               (N_PROF, PRES_INTERPOLATED) float32 9MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    PSAL               (N_PROF, PRES_INTERPOLATED) float32 9MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    SA                 (N_PROF, PRES_INTERPOLATED) float64 19MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    SIG0               (N_PROF, PRES_INTERPOLATED) float64 19MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    SPICE              (N_PROF, PRES_INTERPOLATED) float64 19MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    TEMP               (N_PROF, PRES_INTERPOLATED) float32 9MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    sample_rate        (N_PROF, PRES_INTERPOLATED) float32 9MB dask.array<chunksize=(256, 256), meta=np.ndarray>
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_by:           amf2288
    Fetched_constraints:  [x=-175.00/-170.00; y=20.00/25.00; z=0.0/2001.0]
    Fetched_date:         2025/01/03
    Fetched_from:         https://data-argo.ifremer.fr
    Fetched_uri:          https://data-argo.ifremer.fr/dac/aoml/4900313/49003...
    Processing_history:   [PRES,TEMP,PSAL] real-time and adjusted/delayed var...